# Module 1: Evaluation
Deadline: Wednesday, January 17th, midnight (23:59)<br>
Collaboration: This is a team-based assignment. Form teams of 3 people.<br>
Submit: An IPython Notebook for both the theoretical and the experimental parts with the necessary (a) answers, (b) implementation, (c) explanations,  (d) comments, and (e) analysis. Code quality, informative comments, detailed explanations of what each block in the notebook does and more important convincing analysis of the results will be considered when grading.<br>
Submit your work through Blackboard. All students in a team need to submit their work. <br>
Filename: <student 1 id>–<student 2 id>–<student 3 id>–hw1.ipynb <br>
The homework will cover the following three topics covered in Lecture 1, 2, and 3:<br>
Evaluation measures;<br>
Interleaving;<br>
Click models.<br>
The homework has a small theoretical part and a large experimental part.


# Theoretical part

## Hypothesis Testing – The problem of multiple comparisons [5 points]

Experimentation in AI often happens like this:<br>
A. Modify/Build an algorithm<br>
B. Compare the algorithm to a baseline by running a hypothesis test.<br>
C. If not significant, go back to step A<br>
D. If significant, start writing a paper.<br>

Compute the probabilities below (with Type I error for each test = α):<br>
P(mth experiment gives significant result | m experiments lacking power to reject H0)?

If you mean that m-1 experiments have not given significance and the mth has, then: ((1-α)^(m-1))α<br>
If you mean that m experiments have not given significance and the m+1th has, then: ((1-α)^(m))α

P(at least one significant result | m experiments lacking power to reject H0)?

(1-α)^m

# Bias and unfairness in Interleaving experiments [10 points]¶

Balance interleaving has been shown to be biased in a number of corner cases. An example was given during the lecture with two ranked lists of length 3 being interleaved, and a randomly clicking population of users that resulted in algorithm A winning ⅔ of the time, even though in theory the percentage of wins should be 50% for both algorithms. Can you come up with a situation of two ranked lists of length 3 and a distribution of clicks over them for which Team-draft interleaving is unfair to the better algorithm?

Let's assume that A is the better algorithm here.

Algorithm A &nbsp; &nbsp;   Algorithm B<br>
Link 2   &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;   Link 3<br>
Link 3   &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;   Link 4<br>
Link 4   &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;   Link 5<br>

In lists with a distribution like this, the full search page list could be biased to Algorithm B; if B starts the second round, the resulting list would look like this:

B: Link 3<br>
A: Link 2<br>
B: Link 4<br>
B: Link 5<br>

Therefore Algorithm B would win 3/4th of all random clicks, resulting in an bias pro algorithm B.

# Experimental part
Commercial search engines use both offline and online approach in evaluating a new search algorithm: they first use an offline test collection to compare the production algorithm (P) with the new experimental algorithm (E); if E statistically significantly outperforms P with respect to the evaluation measure of their interest, the two algorithms are then compared online through an interleaving experiment.<br>
For the purpose of this homework we will assume that the evaluation measures of interest are:<br>
1. Binary evaluation measures<br>
a. Precision at rank k,<br>
b. Recall at rank k,<br>
c. Average Precision,<br>
2. Multi-graded evaluation measures<br>
a. Normalized Discounted Cumulative Gain at rank k (nDCG@k),<br>
b. Expected Reciprocal Rank (ERR).<br>

Further, for the purpose of this homework we will assume that the interleaving algorithms of interest are:
1. Team-Draft Interleaving (Joachims. "Evaluating retrieval performance using clickthrough data". Text Mining 2003.)
2. <strike>Probabilistic Interleaving (Hofmann, Whiteson, and de Rijke. "A probabilistic method for inferring preferences from clicks." CIKM 2011.).</strike>
 
In an interleaving experiment the ranked results of P and E (against a user query) are interleaved in a single ranked list which is presented to a user. The user then clicks on the results and the algorithm that receives most of the clicks wins the comparison. The experiment is repeated for a number of times (impressions) and the total wins for P and E are computed. 

A Sign/Binomial Test is then run to examine whether the difference in wins between the two algorithms is statistically significant (or due to chance). Alternatively one can calculate the proportion of times the E wins and test whether this proportion, p, is greater than p0=0.5. This is called an 1-sample 1-sided proportion test.

One of the key questions however is <b>whether offline evaluation and online evaluation outcomes agree with each other</b>. In this homework you will determine the degree of agreement between offline evaluation measures and interleaving outcomes, by the means of simulations. A similar analysis using actual online data can be found at Chapelle et al. “Large-Scale Validation and Analysis of Interleaved Search Evaluation”.

based on lecture 1
## Step 1: Simulate Rankings of Relevance for E and P (5 points)
In the first step you will generate pairs of rankings of relevance, for the production P and experimental E, respectively, for a hypothetical query q. Assume a 3-graded relevance, i.e. {N, R, HR}. Construct all possible P and E ranking pairs of length 5. This step should give you about.

In [1]:
import itertools
import math

possible_rankings = []

possible_evals = 3

for i in range(possible_evals):
    for j in range(possible_evals):
        for k in range(possible_evals):
            for l in range(possible_evals):
                for m in range(possible_evals):
                    new_list = [i, j, k, l, m]
                    possible_rankings.append(new_list)

p_rankings = possible_rankings
e_rankings = possible_rankings

possible_combinations = []
for comb in itertools.combinations(possible_rankings, 2):
    possible_combinations.append(comb)

# Step 2: Implement Evaluation Measures (10 points)
Implement 1 binary and 2 multi-graded evaluation measures out of the 7 measures mentioned above. 
(Note 2: Some of the aforementioned measures require the total number of relevant and highly relevant documents in the entire collection – pay extra attention on how to find this)

In [2]:
def precision(ranking, rank):
    relevant = 0
    for i in range(rank):
        if ranking[i] > 0:
            relevant += 1
    return relevant /rank

def NDCG(ranking, rank):
    total_dcg = 0
    total_ideal_dcg = 0

    for i in range(rank):
        total_dcg += ((2**ranking[i])-1)/math.log2(1+i+1)
        ideal_ranking = sorted(ranking, reverse=True)
        total_ideal_dcg += ((2**ideal_ranking[i])-1)/math.log2(1+i+1)
    
    try:
        result = total_dcg/total_ideal_dcg
    except:
        result = 0
    return result

def ERR(ranking, rank):
    p = 1
    ERR = 0
    for i in range(rank):
        R = satisfy_prob(ranking[i])
        ERR += p * R / (i+1)
        p *= (1-R)
    return ERR

def satisfy_prob(relevance):
    max_relevant = 2
    return (2**relevance - 1)/2**max_relevant  #so probability is always smaller than 1, max 3/4
    

In [3]:
p_precision = precision(p_rankings[10], 5)
print(p_precision)

p_ndcg = NDCG(p_rankings[10], 5)
print(p_ndcg)

p_err = ERR(p_rankings[10], 5)
print(p_err)

test = ([0, 0, 0, 2, 2], [0, 0, 1, 0, 1])

p_precision = precision(test[0], 5)
e_precision = precision(test[1], 5)

p_ndcg = NDCG(test[0], 5)
e_ndcg = NDCG(test[1], 5)

p_err = ERR(test[0], 5)
e_err = ERR(test[1], 5)

print(p_precision, p_ndcg, p_err)
print(e_precision, e_ndcg, e_err)

0.4
0.5437713091520254
0.12083333333333332
0.4 0.5012658353418872 0.225
0.4 0.5437713091520254 0.12083333333333332


# Step 3: Calculate the 𝛥measure (0 points)
For the three measures and all P and E ranking pairs constructed above calculate the difference: 𝛥measure = measureE-measureP. Consider only those pairs for which E outperforms P.


In [4]:
good_precision = []
good_NDCG = []
good_ERR = []
good_combinations = []

for comb in possible_combinations:
    
    p_precision = precision(comb[0], 5)
    e_precision = precision(comb[1], 5)
    
    if(e_precision > p_precision):
        good_precision.append(e_precision)
    
    p_ndcg = NDCG(comb[0], 5)
    e_ndcg = NDCG(comb[1], 5)
    
    if(e_ndcg > p_ndcg):
        good_NDCG.append(e_ndcg)
      
    p_err = ERR(comb[0], 5)
    e_err = ERR(comb[1], 5)
    
    if(e_err > p_err):
        good_ERR.append(e_err)    
    
    p_total = p_precision + p_ndcg + p_err
    e_total = e_precision + e_ndcg + e_err
    
    if(e_total > p_total):
        good_combinations.append(comb)
        
print("There are %d out of %d combinations found where the precision for that combination is better for E than for P." % (len(good_precision), (len(possible_combinations))))
print("There are %d out of %d combinations found where the NDCG score for that combination is better for E than for P." % (len(good_NDCG), (len(possible_combinations))))
print("There are %d out of %d combinations found where the ERR score for that combination is better for E than for P." % (len(good_ERR), (len(possible_combinations))))
print("There are %d out of %d combinations found where the combined score over all three measures for that combination is better for E than for P." % (len(good_combinations), (len(possible_combinations))))

There are 16286 out of 29403 combinations found where the precision for that combination is better for E than for P.
There are 25931 out of 29403 combinations found where the NDCG score for that combination is better for E than for P.
There are 27868 out of 29403 combinations found where the ERR score for that combination is better for E than for P.
There are 26576 out of 29403 combinations found where the combined score over all three measures for that combination is better for E than for P.


based on lecture 2
# Step 4: Implement Interleaving (15 points)
Implement 2 interleaving algorithms: (1) Team-Draft Interleaving OR Balanced Interleaving, <strike>AND (2), Probabilistic Interleaving</strike>. The interleaving algorithms should (a) given two rankings of relevance interleave them into a single ranking, and (b) given the users clicks on the interleaved ranking assign credit to the algorithms that produced the rankings.

(Note 4: Note here that as opposed to a normal interleaving experiment where rankings consists of urls or docids, in our case the rankings consist of relevance labels. Hence in this case (a) you will assume that E and P return different documents, (b) the interleaved ranking will also be a ranking of labels.)


In [5]:
import random

def balanced_interleave(P, E):
    Kp = 0
    Ke = 0
    result = []
    p_first = False
    coin = random.uniform(0,1)
    if coin > .5:
        p_first = True
    
    while Kp < len(P) or Ke < len(E):
        if Kp < Ke or (Kp == Ke and p_first):
            result.append({"rel": P[Kp], "source": "P"})
            Kp += 1
        else:
            result.append({"rel": E[Ke], "source": "E"})
            Ke += 1
    return result

In [6]:
rankings = []
for comb in good_combinations:
    result = balanced_interleave(comb[0], comb[1])
    rankings.append(result)

based on lecture 3
# Step 5: Implement User Clicks Simulation (15 points)
Having interleaved all the ranking pairs an online experiment could be ran. However, given that we do not have any users (and the entire homework is a big simulation) we will simulate user clicks.
We have considered a number of click models including:<br />
- Random Click Model (RCM)<br />
- Position-Based Model (PBM)<br />
- Simple Dependent Click Model (SDCM)<br />
- Simple Dynamic Bayesian Network (SDBN)<br />

Consider two different click models, (a) the Random Click Model (RCM), and (b) one out of the remaining 3 aforementioned models. The parameters of some of these models can be estimated using the Maximum Likelihood Estimation (MLE) method, while others require using the Expectation-Maximization (EM) method. Implement the two models so that (a) there is a method that learns the parameters of the model given a set of training data, (b) there is a method that predicts the click probability given a ranked list of relevance labels, (c) there is a method that decides - stochastically - whether a document is clicked based on these probabilities.<br />

Having implemented the two click models, estimate the model parameters using the Yandex Click Log 

(Note 6: Do not learn the attractiveness parameter 𝑎_uq)

In [7]:
from collections import Counter

class RCM:
    
    def __init__(self):
        self.p = 0
    
    def train(self, data):
        #p = #clicks / #documents
        documents = 0
        clicks = 0
        for row in data:
            line = row.split()
            if line[2] == "Q":
                documents += len(line)-5
            else:
                clicks += 1
        self.p = clicks/documents
                

class PBM:
    def __init__(self):
        self.p = dict()
        
    def train(self, data):
        documents = Counter()
        clicks = Counter()
        
        for row in data:
            line = row.split()
            if line[2] == "Q":
                cur_urls = dict()
                for i in range(0, len(line)-5):
                    cur_urls[i] = line[i+5]
                    documents[i] += 1
            else:
                url_click = line[3]
                for pos, url in cur_urls.items():
                    if url_click == url:
                        break
                clicks[pos] += 1
        for pos, document_count in documents.items():
            self.p[pos] = clicks[pos]/document_count

In [8]:
random_click_model = RCM()
position_click_model = PBM()

with open('YandexRelPredChallenge.txt') as f:
    random_click_model.train(f)

with open('YandexRelPredChallenge.txt') as f:
    position_click_model.train(f)
    
print(random_click_model.p)
print(position_click_model.p)


0.13445559411047547
{0: 0.48844133921035354, 1: 0.20608646722310794, 2: 0.14632373628434775, 3: 0.11019412923192347, 4: 0.08529494513739098, 5: 0.07064147050548626, 6: 0.06203695020163181, 7: 0.055097064615961734, 8: 0.05104098283785051, 9: 0.06939885585670075}


# Step 6: Simulate Interleaving Experiment (10 points)
Having implemented the click models, it is time to run the simulated experiment.
For each of interleaved ranking run N simulations for each one of the click models implemented and measure the proportion p of wins for E.

(Note 7: Some of the models above include an attractiveness parameter 𝑎uq. Use the relevance label to assign this parameter by setting 𝑎uq for a document u in the ranked list accordingly. (See Click Models for Web Search)

In [9]:
def simulate_random_model(ranking, N):
    e_wins = 0
    for i in range(N):
        for url in ranking:
            coin = random.uniform(0,1)
            if coin < random_click_model.p and url["source"] == "E":
                e_wins += 1
    return e_wins / N

def simulate_position_model(ranking, N):
    e_wins = 0
    for i in range(N):
        for j in range(len(ranking)):
            coin = random.uniform(0,1)
            if coin < position_click_model.p[j]*ranking[j]["rel"] and ranking[j]["source"] == "E":
                e_wins += 1
    return e_wins / N

In [11]:
print(rankings[2000])
print(simulate_random_model(rankings[2000], 1000))
print(simulate_position_model(rankings[2000], 1000))

[{'rel': 0, 'source': 'P'}, {'rel': 1, 'source': 'E'}, {'rel': 0, 'source': 'P'}, {'rel': 1, 'source': 'E'}, {'rel': 0, 'source': 'P'}, {'rel': 1, 'source': 'E'}, {'rel': 2, 'source': 'P'}, {'rel': 1, 'source': 'E'}, {'rel': 2, 'source': 'P'}, {'rel': 2, 'source': 'E'}]
0.686
0.589


# Step 7: Results and Analysis (30 points)
Compare the results of the offline experiments (i.e. the values of the 𝛥measure) with the results of the online experiment (i.e. proportion of wins), analyze them and reach your conclusions regarding their agreement.
Use easy to read and comprehend visuals to demonstrate the results;
Analyze the results on the basis of:
- the evaluation measure used,
- the interleaving method used,
- the click model used.

Report and ground your conclusions.

(Note 8: This is the place where you need to demonstrate your deeper understanding of what you have implemented so far; hence the large number of points assigned. Make sure you clearly do that so that the examiner of your work can grade it accordingly.)


# Extra info
Yandex Click Log File:
The dataset includes user sessions extracted from Yandex logs, with queries, URL rankings and clicks. To allay privacy concerns the user data is fully anonymized. So, only meaningless numeric IDs of queries, sessions, and URLs are released. The queries are grouped only by sessions and no user IDs are provided. The dataset consists of several parts. Logs represent a set of rows, where each row represents one of the possible user actions: query or click.
In the case of a Query:
SessionID TimePassed TypeOfAction QueryID RegionID ListOfURLs


In the case of a Click:
SessionID TimePassed TypeOfAction URLID


SessionID - the unique identifier of the user session.
TimePassed - the time elapsed since the beginning of the current session in standard time units.
TypeOfAction - type of user action. This may be either a query (Q), or a click (C).
QueryID - the unique identifier of the request.
RegionID - the unique identifier of the country from which a given query. This identifier may take four values.
URLID - the unique identifier of the document.
ListOfURLs - the list of documents from left to right as they have been shown to users on the page extradition Yandex (top to bottom).